## Homework 6 (due 23:59, 18/Aug/2018)

Use the SPAM SMS messages dataset from [here](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection).

- (1 point) Read the data, get percentage of SPAM and non-SPAM messages.
- (1 point) Develop an algorithm which will be able to correctly classify SPAM vs not SPAM. What is your final accuracy on the test set?
- (1 point) What are the features (words) that affect most on the decision to classify as SPAM or not SPAM? (Hint: check feature importance for Naive Bayes in google)
- (2 point) What is the probability that the following text is a SPAM? "Congratulations! You won a free mobile phone. Go to www.website.am to claim your reward or just reply by text"


In [236]:
import numpy as np
import pandas as pd

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, AgglomerativeClustering

import csv

from sklearn.naive_bayes import GaussianNB 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [108]:
with open('SMSSpamCollection.txt', 'r', encoding='utf-8') as file:
    base_data = file.readlines()

In [109]:
type(base_data)

list

In [110]:
print(base_data[:5])

['ham\tGo until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...\n', 'ham\tOk lar... Joking wif u oni...\n', "spam\tFree entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's\n", 'ham\tU dun say so early hor... U c already then say...\n', "ham\tNah I don't think he goes to usf, he lives around here though\n"]


In [111]:
for i in range(len(base_data)):
    base_data[i] = base_data[i].split('\t')
    base_data[i][1] = base_data[i][1].replace('\n', '')
spam = np.array(base_data)

In [112]:
spam_df = pd.DataFrame(base_data, columns=['type', 'text'])
spam_df.head()

,type,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [114]:
spam_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5574 entries, 0 to 5573
Data columns (total 2 columns):
type    5574 non-null object
text    5574 non-null object
dtypes: object(2)
memory usage: 87.2+ KB


In [116]:
spam_df.to_csv('spam.csv')

#### Read the data, get percentage of SPAM and non-SPAM messages.

In [117]:
spam_df.groupby('type').size()

type
ham     4827
spam     747
dtype: int64

In [118]:
ham_count = spam_df.groupby('type').size()[0]
spam_count = spam_df.groupby('type').size()[1]

In [119]:
def get_percentage(ham, spam):
    """Takes counts of hams and spams, and gives %-s."""
    total = sum([ham, spam])
    perc_ham = (ham/total) * 100
    perc_spam = (spam/total) * 100
    print("Total:", total, " Ham:", ham, " Spam:", spam)
    print("Percentage of hams in total: %.2f" % perc_ham)
    print("Percentage of spams in total: %.2f" % perc_spam)  
    return (perc_ham, perc_spam)

In [120]:
perc_ham, perc_spam = get_percentage(ham_count, spam_count)

Total: 5574  Ham: 4827  Spam: 747
Percentage of hams in total: 86.60
Percentage of spams in total: 13.40


#### Develop an algorithm which will be able to correctly classify SPAM vs not SPAM. What is your final accuracy on the test set?

In [135]:
messages = list(spam_df.text)

In [175]:
cv = CountVectorizer()

In [176]:
messages_vec = cv.fit_transform(messages)

In [177]:
type_vec = np.where(np.asarray(list(spam_df.type))=="ham",0,1)

In [178]:
type_vec[:5]

array([0, 0, 1, 0, 0])

In [181]:
y = type_vec
x = messages_vec.toarray()

In [182]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.25)

In [183]:
model = GaussianNB()

In [184]:
model.fit(x_train, y_train)

GaussianNB(priors=None)

In [185]:
model.score(x_train, y_train)

0.945933014354067

In [186]:
model.score(x_test, y_test)

0.9153515064562411

In [207]:
model.predict(x_test)

array([0, 0, 0, ..., 0, 0, 1])

In [237]:
accuracy_score(y_test, model.predict(x_test)) * 100

91.53515064562411

#### Final accuracy for test set: 91.5%

#### What are the features (words) that affect most on the decision to classify as SPAM or not SPAM? (Hint: check feature importance for Naive Bayes in google)

In [325]:
pred_proba = model.predict_proba(x_train)
words = np.take(cv.get_feature_names(), pred_proba.argmax(axis=1))
words.shape

(4180,)

In [331]:
important_terms = words.argsort()[-10:]
key_list = list(cv.vocabulary_.keys())
val_list = list(cv.vocabulary_.values())
for i in important_terms:
    print(key_list[val_list.index(i)])

achieve
abta
accenture
exist
accomodate
accomodations
accordin
accounting
academic
clever


#### What is the probability that the following text is a SPAM? "Congratulations! You won a free mobile phone. Go to www.website.am to claim your reward or just reply by text"

In [408]:
txt = "Congratulations! You won a free mobile phone. Go to www.website.am to claim your reward or just reply by text"

In [432]:
model.predict_proba(cv.fit_transform([txt]).toarray())

ValueError: operands could not be broadcast together with shapes (1,19) (8713,) 